In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

VOCAB_SIZE = 50
EMBED_DIM = 300
ENC_HIDDEN_DIM = 256
ENC_OUTPUT_DIM = 256


In [28]:
class Encoder(nn.Module):

    def __init__(self,vocab_size,embed_dim = EMBED_DIM,enc_hidden_dim = ENC_HIDDEN_DIM,enc_output_dim = ENC_OUTPUT_DIM,NUM_LAYERS=1,dropout=0.3):
        super().__init__()
        self.embedding_layer = nn.Embedding(vocab_size,EMBED_DIM)
        self.rnn = nn.GRU(embed_dim,enc_hidden_dim, num_layers = NUM_LAYERS ,batch_first= True ,bidirectional=True)
        
        self.fc = nn.Linear(2*enc_hidden_dim,enc_output_dim)            
        self.dropout = nn.Dropout(dropout)

    def forward(self,inp,inp_len):
        embedded_input = self.embedding_layer(inp)
        packed_embedding = nn.utils.rnn.pack_padded_sequence(embedded_input,inp_len)
        packed_output , hidden = self.rnn(packed_embedding)
        outputs, _  = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)))
        return outputs,hidden


In [ ]:
class Attention(nn.Module):
    def __init__(self,enc_hidden_dim, dec_hidden_dim):
        super().__init__()
        
        self.attn = nn.Linear(enc_hidden_dim+dec_hidden_dim,dec_hidden_dim)
        self.v = nn.Linear(dec_hidden_dim,1)

    def forward(self,hidden,encoder_outputs):
        
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        h = hidden.unsqueeze(1).repeat(1,src_len,1)
        e = torch.tanh(self.attn(torch.cat((h,encoder_outputs),dim=2)))
        attention_scores = self.v(e).squeeze(2)
        return F.softmax(attention_scores,dim=1)
  


In [ ]:
class Decoder(nn.Module):

    def __init__(self,vocab_size,enc_hidden_dim,dec_hidden_dim,emb_dim):
        super().__init__()
        
        self.attention = Attention(enc_hidden_dim,dec_hidden_dim)
        self.embedding_layer = nn.Embedding(vocab_size,emb_dim)
        self.rnn = nn.GRU(enc_hidden_dim + dec_hidden_dim)


    def forward(self,input,hidden,encoder_outputs):
            # encoder outputs =  batch_size , seq_len , encoder_output_dim
            # hidden = batch_size , hidden_dim
            # input = batch_size
            
            input = input.unsqueeze(0) # [1,batch_size]
            embedded = self.embedding_layer(input) # [1,batch_size,embed_dim]

            attention_vector = self.attention(hidden,encoder_outputs) # [ batch_size , seq_length ]
            attention_vector = attention_vector.unsqueeze(1) # [batch_size , 1 , seq_length ]

            weighted = torch.bmm(attention_vector,encoder_outputs) # [ batch_size, 1, encoder_output_dim]
            weighted = weighted.permute(1,0,2) #[1 , batch_size , encoder_output_dim]


            rnn_input = torch.cat((embedded,weighted),dim=2) #[1, batch_size, encoder + decoder]

            out,hidden = self.rnn(rnn_input,hidden.unsqueeze(0)) # consider only a single layer (1.)

            embedded = embedded.squeeze(0)
            out = out.squeeze(0)
            weighted = weighted.squeeze(0)

            

            







        


In [ ]:
from turtle import forward


class Model(nn.Module):

    def __init__(self,encoder,decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder



    def forward(self,source,target,teacher_forcing_ratio = 0.5):
        #   source = [batch_size, src_len]
        #   target = [batch_size,traget_length]
        #   teacher_forcing_ratio = probability to use teacher forcinbg

        batch_size = source.shape[0]
        target_length = target.shape[1]
        
        pass


In [29]:
rnn = nn.GRU(100,64, num_layers= 2 ,batch_first= True ,bidirectional=True)
inp = torch.randn((2,8,100))
outputs,hidden = rnn(inp)

In [30]:
outputs.shape

torch.Size([2, 8, 128])

In [31]:
hidden.shape

torch.Size([4, 2, 64])

In [26]:
inq = torch.cat([hidden[-i , : , :] for i in range(4)],dim=1)


In [27]:
inq.shape

torch.Size([2, 256])

In [20]:
rnn(inp)[1].shape

torch.Size([4, 2, 64])

In [9]:
embedding_layer = nn.Embedding(10,30)

embedding_layer(torch.tensor([[10,2,3],[4,5,6]])).shape

IndexError: index out of range in self